# NAGuide R methods

Setup basic methods and packages used for all methods

- BiocManager could be moved to methods who are installed from BioConductor

In [1]:
packages_base_R <-
  c("BiocManager", "reshape2", "data.table", "readr", "tibble")

install_rpackage  <- function(pkg) {
  # If not installed, install the package
  if (!require(pkg, character.only = TRUE)) {
    install.packages(pkg)
    library(pkg, character.only = TRUE)
  }
  
}

# used in the large imputation function for two packages
install_bioconductor  <- function(pkg) {
  # If not installed, install the package
  if (!require(pkg, character.only = TRUE)) {
    BiocManager::install(pkg)
    library(pkg, character.only = TRUE)
  }
  
}


for (package in packages_base_R) {
  # Check if the package is already installed
  install_rpackage(pkg = package)
}


Loading required package: BiocManager



Loading required package: reshape2



Loading required package: data.table



Warning message:
“package ‘data.table’ was built under R version 4.3.3”



Attaching package: ‘data.table’




The following objects are masked from ‘package:reshape2’:

    dcast, melt




Loading required package: readr



Warning message:
“package ‘readr’ was built under R version 4.3.2”


Loading required package: tibble



setup can be tricky... trying to integrate as much as possible into conda environment

Copied from [NAGuideR's github](https://github.com/wangshisheng/NAguideR/blob/15ec86263d5821990ad39a8d9f378cf4d76b25fb/inst/NAguideRapp/app.R#L1705-L1849) RShiny application. Adapted to run as standalone function in context of the Snakemake workflow.

- `df` and `df1` ?
- seems quite hacky
- code is only slightly adapted from repo to run here, mainly to install packages on the fly

In [2]:
nafunctions <- function(x, method = "zero") {
  df <- df1 <- as.data.frame(x)
  method <- tolower(method)
  if (method == "zero") {
    df[is.na(df)] <- 0
  }
  else if (method == "minimum") {
    df[is.na(df)] <- min(df1, na.rm = TRUE)
  }
  else if (method == "colmedian") {
    install_rpackage('e1071')
    df <- impute(df1, what = "median")
  }
  else if (method == "rowmedian") {
    install_rpackage('e1071')
    dfx <- impute(t(df1), what = "median")
    df <- t(dfx)
  }
  else if (method == "knn_impute") {
    install_bioconductor('impute')
    data_zero1 <-
      impute.knn(as.matrix(df1),
                 k = 10,
                 rowmax = 1,
                 colmax = 1)#rowmax = 0.9, colmax = 0.9
    df <- data_zero1$data
  }
  else if (method == "seqknn") {
    if (!require(SeqKnn)) {
      install.packages("src/R_NAGuideR/SeqKnn_1.0.1.tar.gz",
                       repos = NULL,
                       type = "source")
      library(SeqKnn)
    }
    df <- SeqKNN(df1, k = 10)
  }
  else if (method == "bpca") {
    install_bioconductor('pcaMethods')
    data_zero1 <-
      pcaMethods::pca(
        as.matrix(df1),
        nPcs = ncol(df1) - 1,
        method = "bpca",
        maxSteps = 100
      )
    df <- completeObs(data_zero1)
  }
  else if (method == "svdmethod") {
    install_bioconductor('pcaMethods')
    data_zero1 <-
      pcaMethods::pca(as.matrix(df1),
                      nPcs = ncol(df1) - 1,
                      method = "svdImpute")
    df <- completeObs(data_zero1)
  }
  else if (method == "lls") {
    install_bioconductor('pcaMethods')
    data_zero1 <- llsImpute(t(df1), k = 10)
    df <- t(completeObs(data_zero1))
  }
  else if (method == "mle") {
    install_rpackage('norm')
    xxm <- as.matrix(df1)
    ss <- norm::prelim.norm(xxm)
    thx <- norm::em.norm(ss)
    norm::rngseed(123)
    df <- norm::imp.norm(ss, thx, xxm)
  }
  else if (method == "qrilc") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    xxm <- t(df1)
    data_zero1 <-
      imputeLCMD::impute.QRILC(xxm, tune.sigma = 1)[[1]]
    df <- t(data_zero1)
  }
  else if (method == "mindet") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    xxm <- as.matrix(df1)
    df <- imputeLCMD::impute.MinDet(xxm, q = 0.01)
  }
  else if (method == "minprob") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    xxm <- as.matrix(df1)
    df <-
      imputeLCMD::impute.MinProb(xxm, q = 0.01, tune.sigma = 1)
  }
  else if (method == "irm") {
    install_rpackage('VIM')
    df <- irmi(df1, trace = TRUE, imp_var = FALSE)
    rownames(df) <- rownames(df1)
  }
  else if (method == "impseq") {
    install_rpackage('rrcovNA')
    df <- impSeq(df1)
  }
  else if (method == "impseqrob") {
    install_rpackage('rrcovNA')
    data_zero1 <- impSeqRob(df1, alpha = 0.9)
    df <- data_zero1$x
  }
  else if (method == "mice-norm") {
    install_rpackage('mice')
    minum <- 5
    datareadmi <- mice(df1,
                       m = minum,
                       seed = 1234,
                       method = "norm")
    newdatareadmi <- 0
    for (i in 1:minum) {
      newdatareadmi <- complete(datareadmi, action = i) + newdatareadmi
    }
    df <- newdatareadmi / minum
    rownames(df) <- rownames(df1)
  }
  else if (method == "mice-cart") {
    install_rpackage('mice')
    minum <- 5
    datareadmi <- mice(df1,
                       m = minum,
                       seed = 1234,
                       method = "cart")
    newdatareadmi <- 0
    for (i in 1:minum) {
      newdatareadmi <- complete(datareadmi, action = i) + newdatareadmi
    }
    df <- newdatareadmi / minum
    rownames(df) <- rownames(df1)
  }
  else if (method == "trknn") {
    source('src/R_NAGuideR/Imput_funcs.r')
    # sim_trKNN_wrapper <- function(data) {
    #   result <- data %>% as.matrix %>% t %>% imputeKNN(., k=10, distance='truncation', perc=0) %>% t
    #   return(result)
    # }
    # df1x <- sim_trKNN_wrapper(t(df1))
    # df<-as.data.frame(t(df1x))
    df <-
      imputeKNN(as.matrix(df),
                k = 10,
                distance = 'truncation',
                perc = 0)
    df <- as.data.frame(df)
  }
  else if (method == "rf") {
    install_rpackage("missForest")
    data_zero1 <- missForest(
      t(df1),
      maxiter = 10,
      ntree = 20 # input$rfntrees
      ,
      mtry = floor(nrow(df1) ^ (1 / 3)),
      verbose = TRUE
    )
    df <- t(data_zero1$ximp)
  }
  else if (method == "pi") {
    width <- 0.3 # input$piwidth
    downshift <- 1.8 # input$pidownshift
    for (i in 1:ncol(df1)) {
      temp <- df1[[i]]
      if (sum(is.na(temp)) > 0) {
        temp.sd <- width * sd(temp[!is.na(temp)], na.rm = TRUE)
        temp.mean <-
          mean(temp[!is.na(temp)], na.rm = TRUE) - downshift * sd(temp[!is.na(temp)], na.rm = TRUE)
        n.missing <- sum(is.na(temp))
        temp[is.na(temp)] <-
          rnorm(n.missing, mean = temp.mean, sd = temp.sd)
        df[[i]] <- temp
      }
    }
    df
  }
  # else if(method=="grr"){
  #   library(DreamAI)
  #   df<-impute.RegImpute(data=as.matrix(df1), fillmethod = "row_mean", maxiter_RegImpute = 10,conv_nrmse = 1e-03)
  # }
  else if (method == "gms") {
    # install.packages('GMSimpute')
    if (!require(GMSimpute)) {
      install.packages(
        "src/R_NAGuideR/GMSimpute_0.0.1.1.tar.gz",
        repos = NULL,
        type = "source"
      )
      
      library(GMSimpute)
    }
    
    df <- GMS.Lasso(df1,
                    nfolds = 3,
                    log.scale = FALSE,
                    TS.Lasso = TRUE)
  }
  else if (method == "msimpute") {
    install_bioconductor("msImpute")
    df <- msImpute(as.matrix(df),
                   method = 'v2')
    df <- as.data.frame(df)
  }
  else if (method == "msimpute_mnar") {
    install_bioconductor("msImpute")
    df <-
      msImpute(as.matrix(df),
               method = 'v2-mnar',
               group = rep(1, dim(df)[2]))
    df <- as.data.frame(df)
  }
  else if (method == "gsimp") {
    options(stringsAsFactors = F)
    # dependencies parly for sourced file
    
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    install_rpackage("magrittr")
    install_rpackage("glmnet")
    install_rpackage("abind")
    install_rpackage("foreach")
    install_rpackage("doParallel")
    source('src/R_NAGuideR/GSimp.R')
    
    # wrapper function with data pre-processing
    pre_processing_GS_wrapper <- function(data_raw_log) {
      # samples in rows, features in columns #
      # Initialization #
      data_raw_log_qrilc <- as.data.frame(data_raw_log) %>%
        impute.QRILC() %>% extract2(1)
      # Centralization and scaling #
      data_raw_log_qrilc_sc <-
        scale_recover(data_raw_log_qrilc, method = 'scale')
      # Data after centralization and scaling #
      data_raw_log_qrilc_sc_df <- data_raw_log_qrilc_sc[[1]]
      # Parameters for centralization and scaling (for scaling recovery) #
      data_raw_log_qrilc_sc_df_param <- data_raw_log_qrilc_sc[[2]]
      # NA position #
      NA_pos <- which(is.na(data_raw_log), arr.ind = T)
      # NA introduced to log-scaled-initialized data #
      data_raw_log_sc <- data_raw_log_qrilc_sc_df
      data_raw_log_sc[NA_pos] <- NA
      # Feed initialized and missing data into GSimp imputation #
      result <-
        data_raw_log_sc %>% GS_impute(
          .,
          iters_each = 50,
          iters_all = 10,
          initial = data_raw_log_qrilc_sc_df,
          lo = -Inf,
          hi = 'min',
          n_cores = 1,
          imp_model = 'glmnet_pred'
        )
      data_imp_log_sc <- result$data_imp
      # Data recovery #
      data_imp <- data_imp_log_sc %>%
        scale_recover(., method = 'recover',
                      param_df = data_raw_log_qrilc_sc_df_param) %>%
        extract2(1)
      return(data_imp)
    }
    df <- t(df) # samples in rows, feature in columns
    df <- pre_processing_GS_wrapper(df)
    df <- t(df) # features in rows, samples in columns
    
  }
  else{
    stop(paste("Unspported methods so far: ", method))
  }
  df <- as.data.frame(df)
  df
}

## Parameters

Choose one of the available methods. 
Some methods might fail for your dataset for unknown reasons
(and the error won't always be easy to understand)
```method
method = 'ZERO'
method = 'MINIMUM'
method = 'COLMEDIAN'
method = 'ROWMEDIAN'
method = 'KNN_IMPUTE'
method = 'SEQKNN'
method = 'BPCA'
method = 'SVDMETHOD'
method = 'LLS'
method = 'MLE'
mehtod = 'LLS'
method = 'QRILC'
method = 'MINDET'
method = 'MINPROB'
method = 'IRM'
method = 'IMPSEQ'
method = 'IMPSEQROB'
method = 'MICE-NORM'
method = 'MICE-CART'
method = 'RF'
method = 'PI'
method = 'GMS'
method = 'TRKNN',
method = 'MSIMPUTE'
method = 'MSIMPUTE_MNAR'
method = 'GSIMP'
```

In [3]:
train_split = 'runs/example/data/data_wide_sample_cols.csv' # test
folder_experiment = 'runs/example/'
method = 'KNN_IMPUTE'

In [4]:
# Parameters
train_split = "runs/example/data/data_wide_sample_cols.csv"
method = "QRILC"
folder_experiment = "runs/example"


## Dump predictions

In [5]:
df <-
  utils::read.csv(
    train_split,
    row.names = 1,
    header = TRUE,
    stringsAsFactors = FALSE
  )
df

,X2019_12_18_14_35_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_19_19_48_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_20_14_15_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_27_12_29_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_29_15_06_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_29_18_18_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_02_17_38_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_03_11_17_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_03_16_58_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_03_20_10_Q.Exactive.HF.X.Orbitrap_6070,⋯,X2020_05_20_12_33_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_20_15_35_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_22_14_57_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_22_17_43_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_26_14_20_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_27_13_57_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_28_04_06_Q.Exactive.HF.X.Orbitrap_6070,X2020_06_01_10_22_Q.Exactive.HF.X.Orbitrap_6070,X2020_06_01_15_41_Q.Exactive.HF.X.Orbitrap_6070,X2020_06_02_09_41_Q.Exactive.HF.X.Orbitrap_6070
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAR2,27.24780,27.21067,27.37741,NA,26.51979,26.91696,NA,24.84589,25.97128,26.89654,⋯,25.89322,26.72637,25.69277,25.53641,27.26284,28.29443,28.02290,NA,NA,28.08796
ABCB6,24.47166,23.77505,26.85246,24.04702,NA,NA,NA,NA,NA,NA,⋯,NA,25.64072,24.13414,24.68758,26.63090,27.25845,27.66939,NA,NA,24.14359
ABHD11,25.71406,NA,26.04554,24.78816,NA,NA,NA,NA,NA,NA,⋯,NA,NA,25.06275,NA,25.49146,27.38574,NA,NA,NA,25.99529
ACAD9,28.54226,28.43667,28.90693,27.89394,NA,NA,27.11158,27.67040,28.52798,28.55457,⋯,28.87634,NA,28.46669,28.37484,28.84882,30.67143,30.90806,27.88524,NA,28.92190
ACTL6A,29.33516,29.69167,30.36942,29.35853,30.16387,30.62567,27.46894,29.39041,NA,NA,⋯,29.29566,29.42134,29.23683,29.77700,30.63858,31.67139,32.04505,NA,29.79982,30.72761
ACTN1,30.83726,30.63478,31.25175,29.99604,33.14268,33.36724,31.48448,NA,32.32141,32.44850,⋯,30.56156,30.27420,30.36359,30.60753,30.77203,32.60096,32.83077,32.26064,32.66778,NA
ACTR3,29.71286,30.33958,30.71506,29.68579,30.98509,31.56901,29.71290,30.08702,30.20501,30.47702,⋯,29.41643,29.80215,30.01837,30.13663,31.14047,32.22334,32.35713,30.11148,31.47957,31.18866
ADAM10,26.70187,25.92934,26.40279,NA,25.82435,25.33043,NA,NA,NA,25.10717,⋯,22.39430,25.92316,26.40224,27.51073,28.17255,29.02836,NA,NA,NA,27.32706
AHSA1,31.69603,31.29581,NA,30.88242,31.14486,31.10062,29.22371,30.44352,30.98970,30.45875,⋯,30.21025,31.30411,30.62462,30.71162,NA,NA,32.79021,29.86223,31.12435,31.78367


- `data.frame` does not allow abritary column names, but only valid column names...
- tibbles don't support rownames, and the imputation methods rely on normal `data.frame`s.
Save the header row for later use.

In [6]:
original_header <- colnames(readr::read_csv(
  train_split,
  n_max = 1,
  col_names = TRUE,
  skip = 0
))
feat_name <- original_header[1]
original_header[1:5]

Rows: 1 Columns: 51


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Gene Names
dbl (40): 2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070, 2019_12_19_19_48_Q...
lgl (10): 2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070, 2020_01_02_17_38_Q...



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Gene Names"                                    
[2] "2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070"
[3] "2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070"
[4] "2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070"
[5] "2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070"

Uncomment to test certain methods (only for debugging, as at least one method per package is tested using Github Actions)

In [7]:
# to_test <- c(
# 'ZERO',
# 'MINIMUM',
# 'COLMEDIAN',
# 'ROWMEDIAN',
# 'KNN_IMPUTE',
# 'SEQKNN',
# 'BPCA',
# 'SVDMETHOD',
# 'LLS',
# 'MLE',
# 'LLS',
# 'QRILC',
# 'MINDET',
# 'MINPROB',
# 'IRM',
# 'IMPSEQ',
# 'IMPSEQROB',
# 'MICE-NORM',
# 'MICE-CART',
# 'RF',
# 'PI',
# 'GMS', # fails to install on Windows
# 'TRKNN',
# 'MSIMPUTE'
# 'MSIMPUTE_MNAR'
# 'GSIMP'
# )

# for (method in to_test) {
#     print(method)
#     pred <- nafunctions(df, method)
# }

Impute and save predictions with original feature and column names

In [8]:
pred <- nafunctions(df, method)
pred <- tibble::as_tibble(cbind(rownames(pred), pred))
names(pred) <- original_header
pred

Loading required package: impute



Loading required package: pcaMethods



Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘pcaMethods’”


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org



Bioconductor version 3.18 (BiocManager 1.30.22), R 4.3.1 (2023-06-16)



...

Loading required package: sandwich



Warning message:
“package ‘sandwich’ was built under R version 4.3.2”


Loading required package: norm



This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
and has been superseded by the norm2 package.



Gene Names,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070,2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070,2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070,2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070,2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070,2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070,2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070,2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070,⋯,2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070,2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070,2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070,2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070,2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070,2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070,2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070,2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070,2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070,2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAR2,27.24780,27.21067,27.37741,23.02126,26.51979,26.91696,24.90432,24.84589,25.97128,⋯,25.89322,26.72637,25.69277,25.53641,27.26284,28.29443,28.02290,24.50691,24.52615,28.08796
ABCB6,24.47166,23.77505,26.85246,24.04702,20.36456,20.66579,20.35345,23.00247,20.56112,⋯,20.16663,25.64072,24.13414,24.68758,26.63090,27.25845,27.66939,22.80216,22.37698,24.14359
ABHD11,25.71406,21.81209,26.04554,24.78816,22.96668,20.94337,23.50261,21.88651,23.18653,⋯,21.60211,20.58270,25.06275,22.41214,25.49146,27.38574,22.85498,23.05613,20.40408,25.99529
ACAD9,28.54226,28.43667,28.90693,27.89394,26.56320,25.99888,27.11158,27.67040,28.52798,⋯,28.87634,26.75392,28.46669,28.37484,28.84882,30.67143,30.90806,27.88524,25.94465,28.92190
ACTL6A,29.33516,29.69167,30.36942,29.35853,30.16387,30.62567,27.46894,29.39041,27.28659,⋯,29.29566,29.42134,29.23683,29.77700,30.63858,31.67139,32.04505,26.89042,29.79982,30.72761
ACTN1,30.83726,30.63478,31.25175,29.99604,33.14268,33.36724,31.48448,29.15003,32.32141,⋯,30.56156,30.27420,30.36359,30.60753,30.77203,32.60096,32.83077,32.26064,32.66778,29.15626
ACTR3,29.71286,30.33958,30.71506,29.68579,30.98509,31.56901,29.71290,30.08702,30.20501,⋯,29.41643,29.80215,30.01837,30.13663,31.14047,32.22334,32.35713,30.11148,31.47957,31.18866
ADAM10,26.70187,25.92934,26.40279,24.55876,25.82435,25.33043,23.81179,24.43817,22.93136,⋯,22.39430,25.92316,26.40224,27.51073,28.17255,29.02836,23.62705,23.86972,24.42093,27.32706
AHSA1,31.69603,31.29581,28.89384,30.88242,31.14486,31.10062,29.22371,30.44352,30.98970,⋯,30.21025,31.30411,30.62462,30.71162,28.91820,29.11233,32.79021,29.86223,31.12435,31.78367


In [9]:
pred <- reshape2::melt(pred, id.vars = feat_name)
names(pred) <- c(feat_name, 'Sample ID', method)
pred <- pred[reshape2::melt(is.na(df))['value'] == TRUE, ]
pred

,Gene Names,Sample ID,QRILC
,<chr>,<fct>,<dbl>
21,ARAF,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,22.95774
24,ARMCX3,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,21.58634
29,ATP1A1,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,30.33319
34,BDH2,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,22.74338
42,CASK,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,21.05857
44,CCBL2,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,23.48855
52,CENPH,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,23.11945
61,CLPP,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,24.82196
64,COMMD5,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,21.78101


In [10]:
dim(pred)

[1] 3334    3

In [11]:
fname = file.path(folder_experiment,
                  'preds',
                  paste0('pred_all_', toupper(method), '.csv'))
fname

[1] "runs/example/preds/pred_all_QRILC.csv"

In [12]:
write_csv(pred, path = fname)

Warning message:
“The `path` argument of `write_csv()` is deprecated as of readr 1.4.0.
ℹ Please use the `file` argument instead.”
